In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
HOME_DIR= 'D:/jupyter/tf1/git/'
import sys, os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

LOCAL_PACKAGE_DIR = os.path.abspath(HOME_DIR+'/keras-yolo3')
sys.path.append(LOCAL_PACKAGE_DIR)

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

Using TensorFlow backend.


#### import #### 


In [3]:
LOCAL_PACKAGE_DIR = os.path.abspath("./keras-yolo3/yolo3")

In [4]:
WEIGHT_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/weights.h5')
ANCHOR_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/yolo_anchors.txt')
CLASS_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/helmetnonhelmet_classes.txt')

In [5]:
from yolo import YOLO
#keras-yolo에서 image처리를 주요 PIL로 수행. 
from PIL import Image
sys.path.append(LOCAL_PACKAGE_DIR)

helmetAlarm = YOLO(model_path=WEIGHT_DIR,
            anchors_path=ANCHOR_DIR,
            classes_path=CLASS_DIR)
#_augmented1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
D:\jupyter\tf1\git\keras-yolo3\yolo3\weights.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
TEST_DIR='D:/jupyter/tf1/HelmetAlarm/data/TESTIMAGE'

In [7]:
# img = Image.open(os.path.join(TEST_DIR, 'test19.jpg'))
# detected_img = helmetAlarm.detect_image(img)
# plt.figure(figsize=(8, 8))
# plt.imshow(detected_img)

In [8]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys
import cv2
import numpy as np
import time
import os
import pyautogui

recent_prt_sec=-1
recent_prt_min=-1
location='on Screen'

class ScooterHelmetDetector(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("ScooterHelmetDetector_demo")
        self.setGeometry(150,150,10+800+10+600+15,580)
        self.initUI()

    def initUI(self):
        self.cpt = cv2.VideoCapture(0)
        self.fps=24
        
        _,self.img_o = self.cpt.read()
        self.img_o = cv2.cvtColor(self.img_o,cv2.COLOR_RGB2GRAY)
        cv2.imwrite('img_o.jpg',self.img_o)

        self.cnt=0

        self.frame = QLabel(self)
        self.frame.resize(800,500)
        self.frame.setScaledContents(True)
        self.frame.move(10,10) 

        self.movielabel=QLabel(self)
        self.movielabel.move(5+200,5+125)
        self.movielabel.resize(400,250)
        self.movielabel.setScaledContents(True)
        self.movie = QMovie('logo.gif', QByteArray(), self)
        self.movie.setCacheMode(QMovie.CacheAll)
        self.movielabel.setMovie(self.movie)
        self.movie.start()
        
        
        self.logtitle=QLabel(self)
        self.logtitle.resize(200,25)
        self.logtitle.move(10+800+15,10)
        self.logtitle.setText("탐지 내역")
        self.logtitle.setFont(QFont("맑은 고딕",10,QFont.Bold))
        

        
        self.table=QTableWidget(self)
        self.table.resize(600,520)
        self.table.move(10+800+15,10+25+5)
        self.table.setRowCount(0)
        self.table.setColumnCount(5)
        self.table.setHorizontalHeaderLabels(['날짜','시각','장소','결과','세부 정보'])
        self.table.setColumnWidth(0,self.table.width()*1/10)
        self.table.setColumnWidth(1,self.table.width()*1/10)
        self.table.setColumnWidth(2,self.table.width()*2/10)
        self.table.setColumnWidth(3,self.table.width()*3/10)
        self.table.setColumnWidth(4,self.table.width()*3/10)
        
        
    

        self.btn_on = QPushButton("화면 연결",self)
        self.btn_on.resize(100,25)
        self.btn_on.move(10+100+5,10+500+15)
        self.btn_on.setStyleSheet("background-color:#8FFFF7;" "font-weight: bold;")
        self.btn_on.clicked.connect(self.start)
        
        self.btn_on2 = QPushButton("카메라 연결",self)
        self.btn_on2.resize(100,25)
        self.btn_on2.move(10,10+500+15)
        self.btn_on2.setStyleSheet("background-color:#8FFFF7;"  "font-weight: bold;")
        self.btn_on2.clicked.connect(self.start2)
        
        self.btn_off = QPushButton("중지",self)
        self.btn_off.resize(100,25)
        self.btn_off.move(10+100+5+100+5,10+500+15)
        self.btn_off.setStyleSheet("background-color:#8FFFF7;"  "font-weight: bold;")
        self.btn_off.clicked.connect(self.stop)

        
        self.msg=QLabel(self)
        self.msg.setText("상태 메세지: ")
        self.msg.resize(100,25)
        self.msg.move(10+100+5+100+5+100+15+100+15+15,10+500+15)
        
        self.prt=QLabel(self)
        self.prt.resize(200,25)
        self.prt.move(10+100+5+100+5+100+15+100+15+25+100-20,10+500+15)
        self.prt.setText("대기 중입니다.")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : Green")
        

        self.sldr=QSlider(Qt.Horizontal,self)
        self.sldr.resize(100,25)
        self.sldr.move(10+100+5+100+5+100+15,10+500+15)
        self.sldr.setMinimum(1)
        self.sldr.setMaximum(30)
        self.sldr.setValue(24)
        self.sldr.valueChanged.connect(self.setFps)
        
        self.sldrtxt1=QLabel(self)
        self.sldrtxt1.setText("FPS  : ")
        self.sldrtxt1.resize(100,25)
        self.sldrtxt1.move(10+100+5+100+5+100+35,10+500+10+25+5)
        self.fpstxt=QLabel(self)
        self.fpstxt.move(10+100+5+100+5+100+35+35+10,10+500+10+25+12)
        self.fpstxt.setText(str(self.fps))

        

        
        self.show()

    def setFps(self):
        self.fps = self.sldr.value()
        self.prt.setText("FPS가 "+str(self.fps)+"로 조정되었습니다.")
        self.prt.setStyleSheet("Color : Black")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.fpstxt.setText(str(self.fps))
        self.timer.stop()
        self.timer.start(1000 / self.fps)

  

    def start(self):
        self.prt.setText("화면과 연결 중입니다 . . .")
        self.prt.setStyleSheet("Color : Red")
        global location
        location='on Screen'
        self.movielabel.hide()
        self.codec = cv2.VideoWriter_fourcc(*'XVID')
        self.timer=QTimer()
        self.timer.timeout.connect(self.nextFrameSlot)
        self.timer.start(int(1000/self.fps))
        
        
    def start2(self):
        self.prt.setText("카메라와 연결 중입니다 . . .")
        self.prt.setStyleSheet("Color : Red")
        global location
        location='서울시 도봉구 창5동'
        self.movielabel.hide()
        self.codec = cv2.VideoWriter_fourcc(*'XVID')
        self.timer=QTimer()
        self.timer.timeout.connect(self.nextFrameSlot2)
        self.timer.start(int(1000/self.fps))
        
    def stop(self):
        self.prt.setText("대기 중입니다.")
        self.prt.setStyleSheet("Color : Green")
        self.movielabel.show()
        self.frame.setPixmap(QPixmap.fromImage(QImage()))
        #self.frame.setPixmap(self.icon)
        self.movielabel.setMovie(self.movie)
        self.movie.start()
        self.timer.stop()

    def nextFrameSlot(self):
        self.prt.setText("탐지 중입니다 . . .")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : blue")
        pic = pyautogui.screenshot(region=(0, 0, 1920, 1080))
        cam = np.array(pic)
        img=Image.fromarray(cam)
        detected_image,label=helmetAlarm.detect_image(img)
        result=np.asarray(detected_image)
        cam=result
        img=QImage(cam,cam.shape[1], cam.shape[0],QImage.Format_RGB888) #input, width, heigth, format
        pix=QPixmap.fromImage(img)
        self.frame.setPixmap(pix)
        t=time.localtime()
        if t.tm_sec%3==0:
            self.prt.setText("탐지 중입니다 . ")
        elif t.tm_sec%3==1:
            self.prt.setText("탐지 중입니다 . . ")
        else:
            self.prt.setText("탐지 중입니다 . . .")
        
        if(not(t.tm_sec==recent_prt_sec and t.tm_min == recent_prt_min) and len(label)!=0):
            self.addRow(label)
            
    def nextFrameSlot2(self):
        self.prt.setText("탐지 중입니다 . . .")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : blue")
        _,cam=self.cpt.read() 
        cam=cv2.cvtColor(cam,cv2.COLOR_BGR2RGB) #bgr rgb로 변환
        img=Image.fromarray(cam)
        detected_image,label=helmetAlarm.detect_image(img)
        result=np.asarray(detected_image)
        cam=result
        img=QImage(cam,cam.shape[1], cam.shape[0],QImage.Format_RGB888) #input, width, heigth, format
        pix=QPixmap.fromImage(img)
        self.frame.setPixmap(pix)
        t=time.localtime()
        if t.tm_sec%3==0:
            self.prt.setText("탐지 중입니다 . ")
        elif t.tm_sec%3==1:
            self.prt.setText("탐지 중입니다 . . ")
        else:
            self.prt.setText("탐지 중입니다 . . .")
            
        
        if(not(t.tm_sec==recent_prt_sec and t.tm_min == recent_prt_min) and len(label)!=0):
            self.addRow(label)
        
    def addRow(self,label):
        rowPosition=self.table.rowCount()
        self.table.insertRow(rowPosition)
        t=time.localtime()
        self.table.setItem(rowPosition , 0, QTableWidgetItem("{}-{}-{}".format(t.tm_year, t.tm_mon, t.tm_mday)))
        self.table.setItem(rowPosition , 1, QTableWidgetItem("{}:{}:{}".format(t.tm_hour,t.tm_min,t.tm_sec)))
        self.table.setItem(rowPosition , 2, QTableWidgetItem(location))
        res=""
        detail=""
        if 'helmet' in label:
            res+="착용 : {}건      ".format(len(label['helmet']))
            detail+="helmet {}".format(str(label['helmet']))
        if 'nonhelmet' in label:
            res+="미착용 : {}건  ".format(len(label['nonhelmet']))
            detail+="nonhelmet {}".format(str(label['nonhelmet']))
        self.table.setItem(rowPosition , 3, QTableWidgetItem(res))
        self.table.setItem(rowPosition , 4, QTableWidgetItem(detail))
        self.table.scrollToBottom()
        
        global recent_prt_sec
        recent_prt_sec=t.tm_sec
        global recent_prt_min
        recent_prt_min=t.tm_min
        print(recent_prt_sec,recent_prt_min)


In [9]:
app=QApplication(sys.argv)
w=ScooterHelmetDetector()
sys.exit(app.exec_())

(416, 416, 3)
Found 1 boxes for img
2.9899889
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.62 (11, 0) (529, 480)
{'nonhelmet': [[(11, 0), (529, 480)]]}
0.032778300000003924
47 27
(416, 416, 3)
Found 1 boxes for img
0.03128730000000246
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.67 (13, 0) (525, 480)
{'nonhelmet': [[(13, 0), (525, 480)]]}
0.032803200000003585
48 27
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (13, 0) (525, 480)
{'nonhelmet': [[(13, 0), (525, 480)]]}
0.03295820000001015
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.66 (13, 0) (525, 480)
{'nonhelmet': [[(13, 0), (525, 480)]]}
0.0334876999999949
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.70 (20, 0) (520, 480)
{'nonhelmet': [[(20, 0), (520, 480)]]}
0.032466700000000515
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.61 (6, 0) (533, 480)
{'nonhelmet': [[(6, 0), (533, 480)]]}
0.032760599999988926
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.70 (12, 0) (526, 480)
{'nonhelmet': [[(12, 0), (526, 480)]]}
0.

(416, 416, 3)
Found 1 boxes for img
0.030069799999992597
(416, 416, 3)
Found 1 boxes for img
0.030276600000007647
(416, 416, 3)
Found 1 boxes for img
0.03049589999999114
(416, 416, 3)
Found 1 boxes for img
0.030073400000006245
(416, 416, 3)
Found 1 boxes for img
0.030648999999996818
(416, 416, 3)
Found 1 boxes for img
0.030228199999996264
(416, 416, 3)
Found 1 boxes for img
0.030337900000006357
(416, 416, 3)
Found 1 boxes for img
0.030669500000001904
(416, 416, 3)
Found 1 boxes for img
0.030530900000002248
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (16, 0) (529, 480)
{'nonhelmet': [[(16, 0), (529, 480)]]}
0.03224769999999921
53 27
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (14, 0) (532, 480)
{'nonhelmet': [[(14, 0), (532, 480)]]}
0.04886520000000871
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.62 (15, 0) (532, 480)
{'nonhelmet': [[(15, 0), (532, 480)]]}
0.03236379999999883
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (13, 0) (533, 480)
{'nonhelmet': [[(13, 0), 

Found 1 boxes for img
0.031334299999997484
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (15, 10) (621, 480)
{'nonhelmet': [[(15, 10), (621, 480)]]}
0.03196049999999673
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.80 (0, 0) (635, 480)
{'nonhelmet': [[(0, 0), (635, 480)]]}
0.03186030000000528
57 27
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.79 (2, 0) (630, 480)
{'nonhelmet': [[(2, 0), (630, 480)]]}
0.0328109000000012
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.77 (6, 0) (626, 480)
{'nonhelmet': [[(6, 0), (626, 480)]]}
0.040489699999994855
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.74 (3, 0) (629, 480)
{'nonhelmet': [[(3, 0), (629, 480)]]}
0.03254029999999375
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.64 (5, 0) (626, 480)
{'nonhelmet': [[(5, 0), (626, 480)]]}
0.044899700000001985
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.75 (0, 0) (635, 480)
{'nonhelmet': [[(0, 0), (635, 480)]]}
0.03228630000000976
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.77 (0, 0)

(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (16, 0) (543, 480)
{'nonhelmet': [[(16, 0), (543, 480)]]}
0.03356070000000955
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (16, 0) (543, 480)
{'nonhelmet': [[(16, 0), (543, 480)]]}
0.03236989999999196
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.65 (19, 0) (540, 480)
{'nonhelmet': [[(19, 0), (540, 480)]]}
0.03205889999999556
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.67 (14, 0) (545, 480)
{'nonhelmet': [[(14, 0), (545, 480)]]}
0.032267799999999625
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.63 (15, 0) (543, 480)
{'nonhelmet': [[(15, 0), (543, 480)]]}
0.034012700000005225
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.61 (12, 0) (547, 480)
{'nonhelmet': [[(12, 0), (547, 480)]]}
0.03297820000000229
2 28
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.62 (12, 0) (547, 480)
{'nonhelmet': [[(12, 0), (547, 480)]]}
0.032126300000001606
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.62 (12, 0) (547, 480)
{'nonhelmet': [[(12

Found 1 boxes for img
nonhelmet 0.72 (26, 0) (532, 480)
{'nonhelmet': [[(26, 0), (532, 480)]]}
0.033686700000004066
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.75 (28, 0) (531, 480)
{'nonhelmet': [[(28, 0), (531, 480)]]}
0.03254570000000001
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.75 (28, 0) (531, 480)
{'nonhelmet': [[(28, 0), (531, 480)]]}
0.032622099999997545
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.71 (25, 0) (534, 480)
{'nonhelmet': [[(25, 0), (534, 480)]]}
0.0322470999999922
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.71 (28, 0) (531, 480)
{'nonhelmet': [[(28, 0), (531, 480)]]}
0.03341539999999554
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.71 (28, 0) (531, 480)
{'nonhelmet': [[(28, 0), (531, 480)]]}
0.032355999999992946
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.71 (28, 0) (530, 480)
{'nonhelmet': [[(28, 0), (530, 480)]]}
0.03208269999998947
(416, 416, 3)
Found 1 boxes for img
nonhelmet 0.71 (23, 0) (536, 480)
{'nonhelmet': [[(23, 0), (536, 480)]]}


SystemExit: 0

D:\anaconda3\envs\tf1\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
